In [1]:
import numpy as np
import pandas as pd
import json

In [2]:
# given a df, a column to search, a search string, a column to edit, and a string to edit in,
# return the df with the column edited appropriately
def clean(df, find_str, find_col="city_ascii", edit_str=None, edit_col=None):
    if not edit_str:
        edit_str = find_str
    if not edit_col:
        edit_col = find_col
    df.loc[df[find_col].str.contains(find_str), edit_col] = edit_str
    return df

In [3]:
# read and merge all data tables
numbeo_beer_market = pd.read_table('numbeo_beer_market.txt', names=["rank", "city", "beer_market"], index_col="rank")
numbeo_beer_pub = pd.read_table("numbeo_beer_pub.txt", names=["rank", "city", "beer_pub"], index_col="rank")
numbeo_bread = pd.read_table('numbeo_bread.txt', names=["rank", "city", "bread"], index_col="rank")
numbeo_coffee = pd.read_table("numbeo_coffee.txt", names=['rank', 'city', 'coffee'], index_col="rank")

numbeo = pd.merge(numbeo_beer_market, numbeo_beer_pub, how="outer", on="city")
numbeo = pd.merge(numbeo, numbeo_bread, how="outer", on="city")
numbeo = pd.merge(numbeo, numbeo_coffee, how="outer", on="city")

In [4]:
# read in region, lat/lng, pop data
city_data = pd.read_csv('../city-data/world-cities-data.csv')
city_data['region'] = city_data['region'].str.title()

In [5]:
# corrections
city_data.loc[city_data.country.str.contains("Bahamas"), "country"] = "Bahamas"
city_data.loc[city_data.country.str.contains("Korea, South"), "country"] = "South Korea"

In [6]:
# create a separate country column
split = numbeo['city'].str.split(", ")
numbeo['country'] = split.str[-1]
numbeo['city'] = split.str[0]

In [7]:
# to align with city_data column names
numbeo = numbeo.rename(columns={"city": "city_ascii"})

In [8]:
# corrections
numbeo.city_ascii = numbeo.city_ascii.str.replace("Saint", "St.")
numbeo.loc[numbeo.city_ascii.str.contains("Petersburg") & numbeo.country.str.contains("Russia"), "city_ascii"] = "Saint Petersburg"

numbeo = clean(numbeo, find_str="Tel Aviv")
numbeo = clean(numbeo, find_str="Jeddah")
numbeo = clean(numbeo, find_str="Arhus", edit_str="Aarhus")
numbeo = clean(numbeo, find_str="Freiburg")
numbeo = clean(numbeo, find_str="Calicut")
numbeo = clean(numbeo, find_str="Lucknow")
numbeo = clean(numbeo, find_str="Padova", edit_str="Padua")
numbeo = clean(numbeo, find_str="Astana")
numbeo = clean(numbeo, find_str="The Hague")
numbeo = clean(numbeo, find_str="Krakow")
numbeo = clean(numbeo, find_str="Zaragoza")
numbeo = clean(numbeo, find_str="Seville")
numbeo = clean(numbeo, find_str="Kyiv")
numbeo = clean(numbeo, find_str="Odessa")
numbeo = clean(numbeo, find_str="Novgorod", edit_str="Nizhniy Novgorod")
numbeo = clean(numbeo, find_str="Erbil")

numbeo = clean(numbeo, find_col="country", find_str="Kosovo")


In [9]:
# load in expatistan data
with open("../expatistan/expatistan-data.json") as f:
    data = json.load(f)
expat = pd.json_normalize(data)
expat.columns = ['city_ascii', 'country', 'beer_market', 'bread', 'coffee', 'beer_pub']

expat.head()

,city_ascii,country,beer_market,bread,coffee,beer_pub
0,Kabul,Afghanistan,2.58,0.49,2.77,5.72
1,Tirana,Albania,1.06,0.77,1.55,3.55
2,Algiers,Algeria,0.88,0.23,2.29,2.10
3,Athens,Greece,1.52,0.91,4.70,5.24
4,Buenos Aires,Argentina,0.93,0.48,2.08,1.84


In [10]:
# merge expatistan and numbeo data
on_cols = ['city_ascii', 'country']
n_e = pd.merge(numbeo, expat, how="outer", on=on_cols, suffixes=("_n", "_e"))

In [11]:
# rows that have expatistan but not numbeo data
n_e[n_e["beer_pub_n"].isna() & n_e["bread_n"].isna() & n_e['coffee_n'].isna() & n_e["beer_market_n"].isna()]


,city_ascii,beer_market_n,beer_pub_n,bread_n,coffee_n,country,beer_market_e,bread_e,coffee_e,beer_pub_e
620,Santa Rosa,NaN,NaN,NaN,NaN,Argentina,1.14,0.40,1.15,2.22
621,Santa Clara,NaN,NaN,NaN,NaN,Cuba,1.22,1.50,3.53,5.91
622,St. Petersburg,NaN,NaN,NaN,NaN,United States,2.42,2.26,5.08,4.84


In [12]:
# rows that have numbeo but not expatistan data
n_e[n_e["beer_pub_e"].isna() & n_e["bread_e"].isna() & n_e['coffee_e'].isna() & n_e["beer_market_e"].isna()]


,city_ascii,beer_market_n,beer_pub_n,bread_n,coffee_n,country,beer_market_e,bread_e,coffee_e,beer_pub_e
50,Espoo,2.95,7.25,2.60,4.91,Finland,NaN,NaN,NaN,NaN
76,Brooklyn,2.72,7.00,3.56,4.32,United States,NaN,NaN,NaN,NaN
100,Brampton,2.53,4.33,1.83,2.65,Canada,NaN,NaN,NaN,NaN
101,St. Paul,2.53,5.00,2.75,4.25,United States,NaN,NaN,NaN,NaN
146,Sliema,2.22,3.62,1.70,2.93,Malta,NaN,NaN,NaN,NaN
147,Navi Mumbai,2.21,2.39,0.45,2.29,India,NaN,NaN,NaN,NaN
174,Derby,2.03,4.76,1.46,3.48,United Kingdom,NaN,NaN,NaN,NaN
178,Bali,2.01,2.47,1.57,2.11,Indonesia,NaN,NaN,NaN,NaN
181,Chennai,2.00,2.05,0.50,1.99,India,NaN,NaN,NaN,NaN
203,Lund,1.88,7.74,3.01,4.33,Sweden,NaN,NaN,NaN,NaN


In [13]:
# corrections prior to merging with city_data
corrections = [['Newcastle', "Newcastle"],
              ["Penang", "George Town"],
              ['Heraklion', 'Irakleio'],
              ['Ahmedabad', 'Ahmadabad'],
              ['Patras', 'Patra'],
              ['Pattaya', 'Phatthaya'],
              ['Visakhapatnam', 'Vishakhapatnam'],
              ['Hanover', 'Hannover'],
              ['Yangon', 'Rangoon'],
              ['Rostov', 'Rostov'],
              ["Goa", "Panaji"],
              ['Ain', "Al `Ayn"],
              ["Santa Cruz", "Santa Cruz"],
              ["Ajman", "`Ajman"],
              ['Chittagong', 'Chattogram'],
              ['Macao', 'Macau'],
              ['Marsa', 'La Marsa'],
              ['Astana', 'Nur-Sultan'],
              ["Seville", "Sevilla"],
              ["Zaporizhzhya", "Zaporizhzhia"]]

for find, edit in corrections:
    n_e = clean(n_e, find_str=find, edit_str=edit)

In [14]:
n_e.loc[n_e.city_ascii.str.contains("Rangoon"), "country"] = "Burma"
n_e.loc[n_e.city_ascii.str.contains("Macau"), "country"] = "Macau"

n_e.loc[n_e.country.str.contains("Cz"), "country"] = "Czechia"
n_e.loc[n_e.country.str.contains("Mace"), "country"] = "Macedonia"
n_e.loc[n_e.country.str.contains("Ivory"), "country"] = "Côte D’Ivoire"
# n_e.loc[n_e.country.str.contains("Kosovo"), "country"] = "Kosovo"

In [15]:
cols = ["city_ascii", 'country','region', 'latitude', 'longitude', 'population', 'admin_name']
admin_cols = ['city_ascii', 
              'admin_name', 
              'country', 
              'region', 
              'latitude', 
              'longitude', 
              'population',
              'beer_market_n', 
              'bread_n', 
              'coffee_n', 
              'beer_pub_n', 
              'beer_market_e', 
              'bread_e', 
              'coffee_e', 
              'beer_pub_e', 
            ]

In [16]:
# for US cities, generate a list of unabbreviated state names
def state_unabbr(state):
    abbr = pd.read_table("../city-data/us-state-abbr.tsv", names=['abbr', 'name'], header=0)
    name = abbr.loc[abbr['abbr'] == state, 'name']
    if len(name) > 0:
        return name.values[0]
    else:
        return np.nan

states = []
for i in split:
    if i[-1] == "United States":
        states.append(state_unabbr(i[1]))
        
states.append("Florida")

In [17]:
# separate numbeo into US and non-US cities
# for US cities, add states under admin_name, then merge with city_data latitude, longitude, region, and population
numbeo_us = n_e[n_e['country'] == 'United States']
numbeo_us["admin_name"] = states

numbeo_us = pd.merge(numbeo_us, city_data[cols], how='left', on=['city_ascii', 'country', 'admin_name'])
numbeo_us = numbeo_us.reindex(columns=admin_cols)

/Users/christalee/Library/Python/3.6/lib/python/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [18]:
len(numbeo_us) == len(n_e[n_e['country'] == 'United States'])

True

In [19]:
numbeo_us.tail()

,city_ascii,admin_name,country,region,latitude,longitude,population,beer_market_n,bread_n,coffee_n,beer_pub_n,beer_market_e,bread_e,coffee_e,beer_pub_e
105,Riverside,California,United States,North America,33.9381,-117.3948,2107852.0,NaN,2.65,NaN,NaN,2.48,2.53,4.74,5.68
106,Henderson,Nevada,United States,North America,36.0133,-115.0380,320189.0,NaN,2.48,3.85,NaN,NaN,NaN,NaN,NaN
107,Lincoln,Nebraska,United States,North America,40.8090,-96.6788,289490.0,NaN,2.40,3.55,NaN,2.68,1.53,4.86,4.56
108,Mesa,Arizona,United States,North America,33.4017,-111.7180,518012.0,NaN,NaN,4.40,NaN,NaN,NaN,NaN,NaN
109,St. Petersburg,Florida,United States,North America,27.7931,-82.6652,265351.0,NaN,NaN,NaN,NaN,2.42,2.26,5.08,4.84


In [20]:
# for non-US cities, don't match on admin_name
numbeo_globe = n_e[n_e['country'] != "United States"]

numbeo_globe = pd.merge(numbeo_globe, city_data[cols], how='left', on=['city_ascii', 'country'])
numbeo_globe = numbeo_globe.reindex(columns=admin_cols)

In [21]:
# rows where city & country had more than one match in city_data
x = n_e[n_e['country'] != "United States"]
for city, country in zip(x['city_ascii'], x['country']):
    y = numbeo_globe.loc[(numbeo_globe.city_ascii == city) & (numbeo_globe.country == country)]
    if len(y) != 1:
        print(y)

   city_ascii admin_name country   region  latitude  longitude  population  \
75    Uppsala    Uppsala  Sweden  Europe    59.8498    17.6389    164535.0   
76    Uppsala  Stockholm  Sweden  Europe    59.8601    17.6400    133117.0   

    beer_market_n  bread_n  coffee_n  beer_pub_n  beer_market_e  bread_e  \
75           2.47     2.92      4.11        8.34           1.95     1.52   
76           2.47     2.92      4.11        8.34           1.95     1.52   

    coffee_e  beer_pub_e  
75      4.48         7.0  
76      4.48         7.0  
    city_ascii admin_name country          region  latitude  longitude  \
132    Windsor    Ontario  Canada  North America    42.2833      -83.0   
133    Windsor     Quebec  Canada  North America    45.5667      -72.0   

     population  beer_market_n  bread_n  coffee_n  beer_pub_n  beer_market_e  \
132    276165.0           2.01     2.15      3.27        4.72           2.26   
133      5367.0           2.01     2.15      3.27        4.72           

In [22]:
# dedupe those rows
numbeo_globe = numbeo_globe.drop(index=[76, 133, 309, 388])

In [23]:
lost = numbeo_globe[np.isnan(numbeo_globe['latitude']) | np.isnan(numbeo_globe['population'])]

# for i in range(len(lost)):
#     row = lost.iloc[i]
#     city = row['city_ascii']
#     country = row['country']
#     index = row.name
#     x = city_data[city_data.city.str.contains(city.strip("()")[-4:]) & (city_data['country'] == country)]
#     y = city_data[city_data.city.str.contains(city.strip("()")[:4]) & (city_data['country'] == country)]
#     if len(x) == 1:
#         print(city, x)
# #         lost[lost.index == index] = update_row(city, x).values
#     elif len(y) == 1:
#         print(city, y)
# #         lost[lost.index == index] = update_row(city, y).values

In [28]:
lost

,city_ascii,admin_name,country,region,latitude,longitude,population,beer_market_n,bread_n,coffee_n,beer_pub_n,beer_market_e,bread_e,coffee_e,beer_pub_e
28,Tel Aviv,NaN,Israel,Europe,32.085300,34.781806,451523.0,3.29,2.39,4.21,9.12,2.73,1.52,4.29,8.00
95,Cardiff,NaN,United Kingdom,Europe,51.481655,-3.179193,361469.0,2.28,1.52,3.71,4.76,2.62,1.27,3.53,4.70
105,Sliema,Sliema,Malta,Europe,35.912761,14.502071,16854.0,2.22,1.70,2.93,3.62,NaN,NaN,NaN,NaN
106,Navi Mumbai,NaN,India,Asia & Pacific,19.030826,73.019854,1119477.0,2.21,0.45,2.29,2.39,NaN,NaN,NaN,NaN
134,Bali,NaN,Indonesia,Asia & Pacific,-8.330498,115.090640,4230051.0,2.01,1.57,2.11,2.47,NaN,NaN,NaN,NaN
152,Lund,NaN,Sweden,Europe,55.702930,13.192945,93577.0,1.88,3.01,4.33,7.74,NaN,NaN,NaN,NaN
164,Vijayawada,NaN,India,Asia & Pacific,16.508759,80.618510,1048240.0,1.79,0.54,1.78,1.30,1.90,0.59,1.29,2.68
170,Noida,NaN,India,Asia & Pacific,28.535633,77.391073,642381.0,1.77,0.48,1.97,1.91,NaN,NaN,NaN,NaN
193,Bhubaneswar,NaN,India,Asia & Pacific,20.266777,85.843559,837737.0,1.63,0.54,1.09,2.05,1.60,0.45,2.34,2.19
203,Padua,NaN,Italy,Europe,45.407717,11.873446,210440.0,1.55,1.77,1.70,4.83,1.25,1.87,1.67,5.69


In [25]:
import qwikidata.sparql

def get_city_wikidata(city, country):
    query = """
    SELECT ?city ?cityLabel ?country ?countryLabel ?population
    WHERE
    {
      ?city rdfs:label '%s'@en.
      ?city wdt:P1082 ?population.
      ?city wdt:P17 ?country.
      ?city rdfs:label ?cityLabel.
      ?country rdfs:label ?countryLabel.
      FILTER(LANG(?cityLabel) = "en").
      FILTER(LANG(?countryLabel) = "en").
      FILTER(CONTAINS(?countryLabel, "%s")).
    }
    """ % (city, country)

    res = qwikidata.sparql.return_sparql_query_results(query)
    out = res['results']['bindings'][0]
    return out

In [26]:
def get_region(country):
    region = numbeo_globe.loc[numbeo_globe.country.str.contains(country), "region"]
    return region.dropna().unique()[0]

In [27]:
import geocoder
import time
from requests import Session

with Session() as s:
    for row in lost.values:
        city = row[0]
        country = row[2]
        try:
#             print(city, country)
            lost.loc[lost.city_ascii == city, "region"] = get_region(country)

            latlng = geocoder.osm(city + ", " + country, session=s).latlng
            if latlng:
                lost.loc[lost.city_ascii == city, 'latitude'] = latlng[0]
                lost.loc[lost.city_ascii == city, "longitude"] = latlng[1]
            
            wikidata = get_city_wikidata(city, country)
            lost.loc[lost.city_ascii == city, "population"] = int(wikidata['population']['value'])
        except:
            pass
        time.sleep(1)


/Users/christalee/Library/Python/3.6/lib/python/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [29]:
for i in range(len(lost)):
    row = lost.iloc[i]
    index = row.name
    numbeo_globe[numbeo_globe.index == index] = row.values

In [30]:
n_e_clean = pd.concat([numbeo_us, numbeo_globe])
len(n_e) == len(n_e_clean)

True

In [31]:
numbeo_globe.tail(25)

,city_ascii,admin_name,country,region,latitude,longitude,population,beer_market_n,bread_n,coffee_n,beer_pub_n,beer_market_e,bread_e,coffee_e,beer_pub_e
492,Madurai,Tamil Nādu,India,Asia & Pacific,9.9197,78.1194,1561129.0,NaN,0.52,0.98,1.91,1.55,0.74,2.09,3.26
493,Windhoek,Khomas,Namibia,Africa,-22.5700,17.0836,322500.0,NaN,NaN,NaN,1.32,1.63,0.70,2.36,1.77
494,Aguascalientes,Aguascalientes,Mexico,South/Latin America,21.8760,-102.2960,934424.0,NaN,NaN,2.27,1.26,0.86,1.23,2.67,1.53
495,Davao,Davao,Philippines,Asia & Pacific,7.0667,125.6000,1632991.0,NaN,1.19,2.90,1.25,1.00,0.92,2.92,1.59
496,Abidjan,Abidjan,Côte D’Ivoire,Africa,5.3364,-4.0267,4980000.0,NaN,0.76,2.46,0.92,1.16,0.54,3.48,3.32
497,Markham,Ontario,Canada,North America,43.8767,-79.2633,328966.0,NaN,1.79,2.97,NaN,NaN,NaN,NaN,NaN
498,Al `Ayn,Abū Z̧aby,United Arab Emirates,Middle East,24.2075,55.7447,766936.0,NaN,1.21,5.01,NaN,2.75,1.07,5.73,7.00
499,Kuwait City,Al ‘Āşimah,Kuwait,Middle East,29.3750,47.9800,637411.0,NaN,1.04,4.71,NaN,NaN,0.79,5.23,NaN
500,Sharjah,Ash Shāriqah,United Arab Emirates,Middle East,25.3575,55.3919,1400000.0,NaN,0.93,3.35,NaN,2.61,0.72,6.00,7.00
501,Ad Dammam,Ash Sharqīyah,Saudi Arabia,Middle East,26.4333,50.1000,903312.0,NaN,0.92,3.10,NaN,NaN,0.87,3.78,NaN


In [32]:
# numbeo.to_csv("numbeo.csv")

n_e_clean.index = list(range(len(n_e_clean)))
n_e_clean.to_csv("n_e_clean.csv")
n_e_clean.to_json("n_e_clean.json")